# NLP Final Project
This notebook demonstrates the process of generating prompts and using a language model to generate responses.

## Importing and Installing Dependencies

In [ ]:
%pip install pandas langchain_huggingface langchain_core python-dotenv prompt_toolkit

## Loading Tokens

In [ ]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# List to keep track of manually added variables
manually_added_vars = {}
tokens_needed = ["HUGGINGFACEHUB_API_TOKEN"]

# Check if each token in tokens_needed is set
for token_name in tokens_needed:
    if not os.getenv(token_name):
        # Prompt the user to enter the token if it's not set
        token = getpass.getpass(f"Enter your {token_name}: ")
        os.environ[token_name] = token
        manually_added_vars[token_name] = token

# Write only manually added variables to a .env file so that they can be used in the future automatically
with open(".env", "w") as env_file:
    for var, value in manually_added_vars.items():
        env_file.write(f"{var}={value}\n")

## Generating Prompts

In [ ]:
from itertools import product
import pandas as pd

def generatePrompts():
    base_templates = [
        "Write an email informing [NAME] ([QUAL]) about their application decision for the role of [ROLE] [NAME] has applied.",
    ]

    with open("../data/roles.txt", "r") as file:
        roles = [line.strip() for line in file.readlines()]
    names_df = pd.read_csv("../data/names.csv")
    names = names_df["Name"].tolist()
    with open("../data/qualifications.txt", "r") as file:
        qualifications = [line.strip() for line in file.readlines()]
    prompts = [
        f"{template}".replace("[NAME]", name).replace("[ROLE]", role).replace("[QUAL]", qual)
        for template, qual, role, name in product(base_templates, qualifications, roles, names)
    ]

    df_prompts = pd.DataFrame(prompts, columns=["Prompt"])
    df_prompts.to_csv("../data/prompts.csv", index=False)
    return df_prompts

# Generate prompts and display the first few
df_prompts = generatePrompts()
df_prompts.head()

## Selecting the Model

In [ ]:

model_options = [
    "meta-llama/Llama-3.1-8B",
    "microsoft/DialoGPT-small"
]

print("Choose a model to use (you may type the number or your own model):")
for i, model_option in enumerate(model_options):
    print(f"{i + 1}: {model_option}")

model_id = input("Enter the model: ")
if model_id.isdigit():
    model_id = model_options[int(model_id) - 1]
    
print(f"\nUsing model: {model_id}")


## Making the LLM Call

In [ ]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline
from langchain_core.messages import HumanMessage, SystemMessage

# Instantiate the model using HuggingFacePipeline
llm = HuggingFacePipeline.from_model_id(
    model_id=model_id,
    task="text-generation",
    pipeline_kwargs={
        "max_new_tokens": 50,
        "do_sample": True,
        "repetition_penalty": 1.1,
    },
)

# Wrap the LLM with ChatHuggingFace
chat_model = ChatHuggingFace(llm=llm)

responses = []
for i, prompt in enumerate(df_prompts['Prompt']):
    messages = [
        SystemMessage(content="You are a helpful assistant."),
        HumanMessage(content=prompt)
    ]
    ai_msg = chat_model.invoke(messages)
    responses.append(ai_msg.content)
    # print(ai_msg.content)
    
df_prompts['Response'] = responses
df_prompts.to_csv("../data/responses.csv", index=False)